# `nanogpt` development

## dataset preparation

In [1]:
with open("../data/shakespeare/input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(f"Length of dataset in characters: {len(text)}\n")
print(f"Sample text: \n{text[:1000]}")

Length of dataset in characters: 1115394

Sample text: 
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
spea

In [2]:
characters = sorted(list(set(text)))
vocabulary_size = len(characters)

print("".join(characters))
print(vocabulary_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


## tokenize
Translate individual characters in to integers

In [3]:
stoi = {ch: i for i, ch in enumerate(characters)}
itos = {i: ch for i, ch in enumerate(characters)}

def encode(string: str) -> list[int]:
    """Encode takes a string and outputs a list of integers"""
    return [stoi[c] for c in string]

def decode(encoded: list[int]) -> str:
    """Decode takes a list of integers and outputs a string"""
    return "".join([itos[i] for i in encoded])

encoded = encode("Hello World!")
decoded = decode(encoded)
print(encoded)
print(decoded)

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
Hello World!


In [4]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

Now let's split for training and test

In [5]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [7]:
def get_batch(data: torch.Tensor, batch_size: int = 4, block_size: int = 8) -> tuple[torch.Tensor, torch.Tensor]:
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x, y

In [8]:
xb, yb = get_batch(train_data)

print(xb.shape, xb)
print(yb.shape, yb)

torch.Size([4, 8]) tensor([[ 1, 50, 47, 43,  1, 47, 52,  1],
        [ 1, 54, 50, 39, 41, 43,  6,  1],
        [21, 27, 10,  0, 35, 46, 63,  6],
        [59, 50,  5, 57,  1, 46, 39, 58]])
torch.Size([4, 8]) tensor([[50, 47, 43,  1, 47, 52,  1, 60],
        [54, 50, 39, 41, 43,  6,  1, 51],
        [27, 10,  0, 35, 46, 63,  6,  1],
        [50,  5, 57,  1, 46, 39, 58, 43]])


## Implement bigram language model

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

In [10]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size: int):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx: torch.Tensor, targets: torch.Tensor = None):
        # Logits is a (batch, time, channel) tensor
        # Predict what comes next based on the identity of a single character
        logits = self.token_embedding_table(idx)

        loss = None
        if targets is not None:
            # Reshape logits to match (batch, channel) for cross entropy
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx: torch.Tensor, max_new_tokens: int = 100):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]  # (batch, time) last time step is the prediction for what comes next
            probabilities = F.softmax(logits, dim=-1)  # (batch, time) apply softmax to get probabilities
            idx_next = torch.multinomial(probabilities, num_samples=1)  # (batch, 1)
            idx = torch.cat((idx, idx_next), dim=1)  # (batch, time + 1)
        return idx

In [11]:
model = BigramLanguageModel(vocabulary_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
prediction = decode(model.generate(idx)[0].tolist())
print(prediction)

torch.Size([32, 65])
tensor(4.7057, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [12]:
logits

tensor([[ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [-0.7470, -1.4852,  0.1714,  ...,  2.2019,  2.4498,  0.6347],
        [ 1.6515, -0.0424, -0.7355,  ...,  0.8682,  2.0593, -0.8159],
        ...,
        [ 1.0901,  0.2170, -2.9996,  ..., -0.5472, -0.8017,  0.7761],
        [ 1.1513,  1.0539,  3.4105,  ..., -0.5686,  0.9079, -0.1701],
        [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305]],
       grad_fn=<ViewBackward0>)

In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)

In [14]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch(train_data)
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.420325994491577


In [15]:
idx = torch.zeros((1, 1), dtype=torch.long)
prediction = decode(model.generate(idx, max_new_tokens=1000)[0].tolist())
print(prediction)


LUng:
j?
o f herere s digl y somm d fethal m,
KTIN fl tthane abe fls th un,
AD3XEr tidsethe n, or d GLGHAky!
G&paveelt pofotes dend tou:

AEVuing
SPKI' thickerenlaig, at wg wonofang dstobouThee CH:P! t DI fuf O my me he n:
Biset,Uinl, ke is tor, micuais s.
blt ind'thegandisins m.
W-ves: anyoroup meartold n mend? theral igr

CELK$z; ke me En s HOUpanalaf s yo'se, y h d hd t t.
Uhe dizk:
Clalesed wr't tho aheck, or bbllllf whoul dicy.
I tly d nd kshiseline Mhet wEN FJUpUgathaite, n:wo o-gusatofthind te.
lo anchty ared makin cors as ay, by Hed hoagrpove s, cthd
ou m, wive?.

ENCoros ardash is O:
Ald bimoCPOFVWhspese ts
&!.Vog?dcinely, ivehowemethecad th m;
ONCldlirD.
Yo-labjR:
Win, ceere Imesich bh'me d.
AR:
A:
TEN!$Tad mone INAUFRW:
GSitg the.
R
tanenon.
CEL-ey pxqbouio prLEOJchofarecomond f me ak-
NV
OLlsoun teauCLClandstryb THord ldLUSjuire
pHalane;; Vknl,
THe y nd r mprlat lalalo
nor RFe winatllorkislmye
AGa dheng ba inem e tat f:
ALjdnt tha mitrks ban?reth bes seon:P
Wrchumed th jul

## the mathematical trick in self-attention

In [16]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [36]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t + 1]  # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)
xbow.shape

torch.Size([4, 8, 2])

Use batch matrix multiply to do mean aggregation for preceding tokens

In [37]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x  # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2)

True

Using softmax, we can start with non-zero token affinities

In [38]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x  # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow3)

True

Self attention!


In [54]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

# Let's see a single head perform self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)  # (B, T, 16)
q = query(x)  # (B, T, 16)
wei = q @ k.transpose(-2, -1)  # (B, T, 16) (B, 16, T) -> (B, T, T)


tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

Weights gives us a nice distribution of how much integration to aggregate of any of the information int he past

In [53]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5599, 0.4401, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3220, 0.2016, 0.4764, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1640, 0.0815, 0.2961, 0.4585, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2051, 0.3007, 0.1894, 0.1808, 0.1241, 0.0000, 0.0000, 0.0000],
        [0.0600, 0.1273, 0.0291, 0.0169, 0.0552, 0.7114, 0.0000, 0.0000],
        [0.1408, 0.1025, 0.1744, 0.2038, 0.1690, 0.0669, 0.1426, 0.0000],
        [0.0223, 0.1086, 0.0082, 0.0040, 0.0080, 0.7257, 0.0216, 0.1016]],
       grad_fn=<SelectBackward0>)